# PogNet3

Multi-Vehicle Model

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import os, os.path
import numpy as np
import pickle
import random
from glob import glob
from tqdm import tqdm


"""Change to the data folder"""
new_path = "data/new_train/"

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""

    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, "*"))
        self.pkl_list.sort()

    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, "rb") as f:
            data = pickle.load(f)

        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
train_dataset = ArgoverseDataset(data_path=new_path)

In [3]:
batch_sz = 100


def my_collate_train(batch):
    """collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature]"""

    batch_inp = []
    batch_out = []

    for scene in batch:
        agent = scene["agent_id"]
        target = 0
        for x in range(len(scene["track_id"])):
            if scene["track_id"][x][0] == agent:
                target = x
        inp = [scene["p_in"][target], scene["v_in"][target]]
        out = [scene["p_out"][target], scene["v_out"][target]]
        batch_inp.append(inp)
        batch_out.append(out)

    # scene level #####################
    # batch_inp = []
    # batch_out = []
    # for scene in batch:
    #   mask = scene['car_mask'].flatten()==1
    #   # print(np.count_nonzero(mask))
    #   inp = [scene['p_in'][mask], scene['v_in'][mask]]
    #   out = [scene['p_out'][mask], scene['v_out'][mask]]
    #   batch_inp.append(inp)
    #   batch_out.append(out)
    ####################################

    inp = torch.LongTensor(batch_inp)
    out = torch.LongTensor(batch_out)
    return [inp, out]


def my_collate_train_multiple(batch):
    """collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature]"""
    batch_inp = []
    batch_out = []

    for scene in batch:
        agent = scene["agent_id"]
        target = 0
        for x in range(len(scene["track_id"])):
            if scene["track_id"][x][0] == agent:
                target = x
        inp = [scene["p_in"][target], scene["v_in"][target]]
        out = [scene["p_out"][target], scene["v_out"][target]]

        other_in = np.zeros((5, 2, 19, 2))  # need to permute to 2,5,19,2 later
        other_out = np.zeros((5, 2, 30, 2))

        other_in[0] = inp
        other_out[0] = out

        mask = scene["car_mask"].flatten() == 1
        mask = np.delete(mask, target)
        mask = np.where(mask == True)[0]

        if len(mask) >= 4:
            temp = random.sample(mask.tolist(), 4)
            for i in range(len(temp)):
                other_in[i + 1] = [scene["p_in"][temp[i]], scene["v_in"][temp[i]]]
                other_out[i + 1] = [scene["p_out"][temp[i]], scene["v_out"][temp[i]]]
        else:
            for i in range(len(mask)):
                other_in[i + 1] = [scene["p_in"][mask[i]], scene["v_in"][mask[i]]]
                other_out[i + 1] = [scene["p_out"][mask[i]], scene["v_out"][mask[i]]]
            for i in range(4 - len(mask)):
                other_in[i + 1 + len(mask)] = inp
                other_out[i + 1 + len(mask)] = out

        batch_inp.append(other_in.tolist())
        batch_out.append(other_out.tolist())

    # scene level #####################
    # batch_inp = []
    # batch_out = []
    # for scene in batch:
    #   mask = scene['car_mask'].flatten()==1
    #   # print(np.count_nonzero(mask))
    #   inp = [scene['p_in'][mask], scene['v_in'][mask]]
    #   out = [scene['p_out'][mask], scene['v_out'][mask]]
    #   batch_inp.append(inp)
    #   batch_out.append(out)
    ####################################

    inp = torch.FloatTensor(batch_inp)
    inp = inp.permute(2, 0, 1, 3, 4)  # p/v, batch, cars, points, x/y
    out = torch.FloatTensor(batch_out)
    out = out.permute(2, 0, 1, 3, 4)
    return [inp, out]


def my_collate_val(batch):
    """collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature]"""

    inp = [[scene["p_in"], scene["v_in"]] for scene in batch]
    mask = [scene["car_mask"] for scene in batch]

    inp = torch.FloatTensor(inp)
    mask = torch.FloatTensor(mask)
    return [inp, mask]

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Trajectory(nn.Module):
    def __init__(self):
        super(Trajectory, self).__init__()

        self.p_in = nn.Linear(2, 32)  # N, 19, 2
        self.v_in = nn.Linear(2, 32)  # N, 19, 2

        self.encoder = nn.LSTM(64, 64, 1)  # input 19, N, 64 output 1, N, 64

        self.decoder_p = nn.LSTM(64, 128, 1)  # input 30, N, 64 output 30, N, 128
        self.decoder_v = nn.LSTM(64, 128, 1)  # input 30, N, 64 output 30, N, 128

        self.p_out = nn.Linear(128, 2)
        self.v_out = nn.Linear(128, 2)

    def forward(self, p, v):
        batch = p.shape[0]
        x_p = self.p_in(p)
        x_v = self.v_in(v)

        x = torch.cat((x_p, x_v), dim=2)
        x = x.permute(1, 0, 2)

        _, (state_h, _) = self.encoder(x)

        x = state_h.repeat(30, 1, 1)

        x_p, _ = self.decoder_p(x)
        x_v, _ = self.decoder_v(x)

        x_p = x_p.permute(1, 0, 2)
        x_v = x_v.permute(1, 0, 2)

        x_p = self.p_out(x_p)
        x_v = self.v_out(x_v)

        return x_p, x_v

In [5]:
"""
Multi-Car Training + Validation
Normalization: /4800, /100
"""

batch_sz = 100
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(
    train_loader.dataset, [train_size, val_size]
)

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(
    train_data,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)
val_data = DataLoader(
    val_data,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)
model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters(), lr=0.001)

epoch = 20  # takes around 20 epochs to converge
number = 5  # number of cars in each

for i in range(epoch):

    model.train()
    epoch_loss = 0

    train_loop = tqdm(enumerate(train_data), total=len(train_data))

    for i_batch, sample_batch in train_loop:

        inp, out = sample_batch
        batch = inp.shape[1]

        p_in = (inp[0].reshape(batch * number, 19, 2).to(device)) / 4800.
        v_in = inp[1].reshape(batch * number, 19, 2).to(device) / 100.
        p_out = (out[0].reshape(batch * number, 30, 2).to(device)) /4800.
        v_out = out[1].reshape(batch * number, 30, 2).to(device) / 100.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        v_criteria = nn.MSELoss()
        v_loss = torch.sqrt(v_criteria(pred[1], v_out))

        loss = p_loss + v_loss
        epoch_loss += p_loss

        my_optim.zero_grad()
        loss.backward()
        my_optim.step()

        train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
        train_loop.set_postfix(loss=epoch_loss.item())

    model.eval()
    epoch_loss = 0
    val_loop = tqdm(enumerate(val_data), total=len(val_data))
    for i_batch, sample_batch in val_loop:
        inp, out = sample_batch
        batch = inp.shape[1]

        p_in = (inp[0].reshape(batch * number, 19, 2).to(device)) / 4800.
        v_in = inp[1].reshape(batch * number, 19, 2).to(device) / 100.
        p_out = (out[0].reshape(batch * number, 30, 2).to(device)) / 4800.
        v_out = out[1].reshape(batch * number, 30, 2).to(device) / 100.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        epoch_loss += p_loss

        val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
        val_loop.set_postfix(loss=epoch_loss.item())


LENGTH OF TRAIN LOADER DATASET: 205942
LENGTH OF TRAIN DATA: 164753 
LENGTH OF VAL DATA: 41189


Val.  Epoch [20/20]: 100%|██████████| 412/412 [00:21<00:00, 19.53it/s, loss=0.546]


In [6]:
"""
Multi-Car Training + Validation
Normalization: /2400, /85
"""

batch_sz = 100
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)

train_size = int(0.8 * len(train_loader.dataset))
val_size = len(train_loader.dataset) - train_size
train_data, val_data = torch.utils.data.random_split(
    train_loader.dataset, [train_size, val_size]
)

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data), "\nLENGTH OF VAL DATA:", len(val_data))

train_data = DataLoader(
    train_data,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)
val_data = DataLoader(
    val_data,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)
model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters(), lr=0.001)

epoch = 20  # takes around 20 epochs to converge
number = 5  # number of cars in each

for i in range(epoch):

    model.train()
    epoch_loss = 0

    train_loop = tqdm(enumerate(train_data), total=len(train_data))

    for i_batch, sample_batch in train_loop:

        inp, out = sample_batch
        batch = inp.shape[1]

        p_in = ((inp[0].reshape(batch * number, 19, 2).to(device)) - 2400.) / 2400.
        v_in = inp[1].reshape(batch * number, 19, 2).to(device) / 85.
        p_out = ((out[0].reshape(batch * number, 30, 2).to(device)) - 2400.) / 2400
        v_out = out[1].reshape(batch * number, 30, 2).to(device) / 85.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        v_criteria = nn.MSELoss()
        v_loss = torch.sqrt(v_criteria(pred[1], v_out))

        loss = p_loss + v_loss
        epoch_loss += p_loss

        my_optim.zero_grad()
        loss.backward()
        my_optim.step()

        train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
        train_loop.set_postfix(loss=epoch_loss.item())

    model.eval()
    epoch_loss = 0
    val_loop = tqdm(enumerate(val_data), total=len(val_data))
    for i_batch, sample_batch in val_loop:
        inp, out = sample_batch
        batch = inp.shape[1]

        p_in = ((inp[0].reshape(batch * number, 19, 2).to(device)) - 2400.) / 2400.
        v_in = inp[1].reshape(batch * number, 19, 2).to(device) / 85.
        p_out = ((out[0].reshape(batch * number, 30, 2).to(device)) - 2400.) / 2400.
        v_out = out[1].reshape(batch * number, 30, 2).to(device) / 85.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        epoch_loss += p_loss

        val_loop.set_description(f"Val.  Epoch [{i + 1}/{epoch}]")
        val_loop.set_postfix(loss=epoch_loss.item())


LENGTH OF TRAIN LOADER DATASET: 205942
LENGTH OF TRAIN DATA: 164753 
LENGTH OF VAL DATA: 41189


Val.  Epoch [20/20]: 100%|██████████| 412/412 [00:21<00:00, 19.09it/s, loss=1.05] 


In [ ]:
"""
Multi-Car Training for Submission
Normalization: /4800, /100
"""

batch_sz = 100
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_sz,
    shuffle=False,
    collate_fn=my_collate_train_multiple,
    num_workers=2,
)

print("\nLENGTH OF TRAIN LOADER DATASET:", len(train_loader.dataset))
print("LENGTH OF TRAIN DATA:", len(train_data))

model = Trajectory().to(device)

my_optim = torch.optim.Adam(model.parameters(), lr=0.001)

epoch = 20  # takes around 20 epochs to converge
number = 5  # number of cars in each

for i in range(epoch):

    model.train()
    epoch_loss = 0

    train_loop = tqdm(enumerate(train_loader), total=len(train_loader))

    for i_batch, sample_batch in train_loop:

        inp, out = sample_batch
        batch = inp.shape[1]

        p_in = (inp[0].reshape(batch * number, 19, 2).to(device)) / 4800.
        v_in = inp[1].reshape(batch * number, 19, 2).to(device) / 100.
        p_out = (out[0].reshape(batch * number, 30, 2).to(device)) /4800.
        v_out = out[1].reshape(batch * number, 30, 2).to(device) / 100.

        pred = model(p_in, v_in)

        loss = 0
        p_criteria = nn.MSELoss()
        p_loss = torch.sqrt(p_criteria(pred[0], p_out))

        v_criteria = nn.MSELoss()
        v_loss = torch.sqrt(v_criteria(pred[1], v_out))

        loss = p_loss + v_loss
        epoch_loss += p_loss

        my_optim.zero_grad()
        loss.backward()
        my_optim.step()

        train_loop.set_description(f"Train Epoch [{i + 1}/{epoch}]")
        train_loop.set_postfix(loss=epoch_loss.item())

    model.eval()
    epoch_loss = 0

In [ ]:
model.eval()
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    batch = inp.shape[0]

    p_in = (
        inp[:, 0].reshape(batch * number, 19, 2).to(device)
    ) / 4800.0  # -2400.)/2400.
    v_in = inp[:, 1].reshape(batch * number, 19, 2).to(device) / 100.0
    p_out = (
        out[:, 0].reshape(batch * number, 30, 2).to(device)
    ) / 4800.0  # -2400.)/2400.
    v_out = out[:, 1].reshape(batch * number, 30, 2).to(device) / 100.0

    pred = model(p_in, v_in)
    print(pred[0][9] * 4800, p_out[9] * 4800)
    print(torch.sum(torch.subtract(pred[0][9] * 4800, p_out[9] * 4800)))
    break

In [ ]:
import csv

model.eval()
temp = []

new_path = "data/new_val_in/"
val_dataset = ArgoverseDataset(data_path=new_path)

top = []
top.append("ID")
for i in range(60):
    top.append("v" + str(i + 1))
temp.append(top)

with torch.no_grad():
    for i in val_dataset:
        row = []
        scene = i["scene_idx"]
        agent = i["agent_id"]
        target = 0
        for x in range(len(i["track_id"])):
            if i["track_id"][x][0] == agent:
                target = x

        p_in = torch.LongTensor(i["p_in"])
        v_in = torch.LongTensor(i["v_in"])

        p_in = (p_in.reshape(60, 19, 2).to(device)) / 4800.0
        v_in = v_in.reshape(60, 19, 2).to(device) / 100.0

        pred = model(p_in, v_in)

        pred_out = pred[0] * 4800.0

        output = pred_out[target]

        row.append(scene)
        row = row + torch.flatten(output).cpu().detach().numpy().tolist()
        temp.append(row)

with open("submission9.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerows(temp)